In [24]:
import pandas as pd
import geopandas as gpd
import topojson as tp
import re
import warnings

# read in sales
sales_df = pd.read_csv('Data/GwinnettSales.csv', index_col=0)

# convert pandas to geodataframe
sales_df = gpd.GeoDataFrame(
    sales_df,
    geometry=gpd.points_from_xy(
        sales_df['PropertyLongitude'], sales_df['PropertyLatitude']),
    crs="EPSG:4269"
)

# read in census tracts
gwinnett_ct = gpd.read_file('Data/gwinnett_CTs.geojson')

# pare down the dataframe
gwinnett_ct = gwinnett_ct[[
    'GEOID',
    'Sub_geo',
    'geometry'
]]

# ignore the warnings that come with simplifying geographically
warnings.filterwarnings("ignore", category=RuntimeWarning)

toposimplify = 0.001
gwinnett_simp = tp.Topology(gwinnett_ct, toposimplify=toposimplify).to_gdf()

# export simplified geometry
gwinnett_simp.to_file('Data/gwinnett_CTs_simp.geojson')

# spatial join
sales_joined = sales_df.sjoin(gwinnett_simp, how="left")

sales_joined = sales_joined.drop(columns='index_right')
sales_joined = sales_joined.rename(columns={
    'PropertyLatitude': 'lat',
    'PropertyLongitude': 'long'
})

sales_joined['Sub_geo'] = sales_joined['Sub_geo'].replace({
    'Snellville/S Gwinnett': 'Snellville/S. Gwinnett'
})

sales_joined['price_sf'] = sales_joined['TransferAmount'] / \
    sales_joined['AreaGross']

# only get sales above some dollar threshold
sales_joined = sales_joined[sales_joined['TransferAmount'] > 30000]

# only get sales with a home size
sales_joined = sales_joined[sales_joined['AreaGross'] > 0]

# only keep 2020 sales onward
sales_joined = sales_joined[sales_joined['year'] >= 2020]

sales_joined.to_csv('Data/Gwinnett_19-24.csv', index=False)

print('export complete!')
print(f'dataframe rows: {sales_joined.shape[0]:,}')

sales_joined.head(3)

export complete!
dataframe rows: 66,799


,PropertyAddressFull,RecordingDate,year,month,year-month,TransferAmount,YearBuilt,AreaGross,lat,long,geometry,GEOID,Sub_geo,price_sf
6,4049 RIVERSTONE DR,2023-09-07,2023,9,2023-9,410000.0,1997.0,2640.0,34.038858,-84.105995,POINT (-84.10599 34.03886),13135050244,Suwanee,155.303030
7,1774 LAWRENCEVILLE SUWANEE RD,2022-04-29,2022,4,2022-4,400000.0,1988.0,1512.0,33.992023,-84.034348,POINT (-84.03435 33.99202),13135050553,Lawrenceville,264.550265
8,3542 STRATHMORE DR,2023-08-09,2023,8,2023-8,435000.0,1992.0,2788.0,34.005478,-84.153473,POINT (-84.15347 34.00548),13135050219,Duluth,156.025825
